In [ ]:
!pip install NumPy==1.24.3

In [16]:
# -*- coding: utf-8 -*-
"""Mesh.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/10HMUQeTcNtXySy3CYzsg1DJPbTAsrV2b
"""


from PIL import Image, ImageDraw, ImageFilter
from PIL.ImageTransform import MeshTransform
import numpy as np
import random

use_mask = True
test_coords = False
test_mesh = False

radius = 5
layers = 1
files = 1

import sys
IN_COLAB = 'google.colab' in sys.modules


folder = '/content/' if IN_COLAB else './'
if test_mesh:
  image_path = './grid_image.png'
else:
##  image_path = './Rhythms_Circle_DataReferenceSet_1982_2.png'
  image_path = './abstract_artwork_with_neon1.png'

def make_mask(width=200, height=200, border=5, rectangle=True):

  # Create a new image with a black background
  image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

  # Calculate center coordinates
  center_x = width // 2
  center_y = height // 2

  # Define radius
  radius = 50

  # Create a drawing object
  draw = ImageDraw.Draw(image)

  # Draw a circle with a gradient edge
  shape = draw.rectangle if rectangle else draw.ellipse
  for i in range(border):
    alpha = int(255 * (i / border))  # Calculate alpha value for gradient
    shape(
        (
            center_x - radius + i,
            center_y - radius + i,
            center_x + radius - i,
            center_y + radius - i,
        ),
        fill=(255, 255, 255, alpha),
    )

  # return the image
  return(image)


def draw_mesh(mesh, image):
  """Draws the mesh on the given image.

  Args:
    mesh: A list of tuples, where each tuple is ((bbox_x1, bbox_y1, bbox_x2, bbox_y2), (quad_x1, quad_y1, quad_x2, quad_y2, quad_x3, quad_y3, quad_x4, quad_y4)).
    image: A PIL Image object.
  """
  draw = ImageDraw.Draw(image)

  for bbox, quad in mesh:
    # Draw bounding box in blue
    draw.rectangle(bbox, outline="blue", width=3)

    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    color = (red,green,blue)

    # Draw quad in red
    draw.line((quad[0],quad[1], quad[2],quad[3], quad[4],quad[5], quad[6],quad[7],quad[0],quad[1]), fill=color, width=8)
    print('quad',(quad[0],quad[1], quad[2],quad[3], quad[4],quad[5], quad[6],quad[7]))

  return image

def create_randomized_aligned_mesh(rows, cols, image_width, image_height):
  """Creates a randomized mesh where all edges align.

  Args:
    rows: Number of rows in the mesh.
    cols: Number of columns in the mesh.
    image_width: Width of the image the mesh will be applied to.
    image_height: Height of the image the mesh will be applied to.

  Returns:
    A list of quadrilaterals representing the mesh.
  """

  # Generate random x and y coordinates for the grid lines
  if not test_coords:
    top_x_coords = [0] + sorted([int(random.uniform(0, image_width)) for _ in range(cols - 1)]) + [image_width]
    bottom_x_coords = [0] + sorted([int(random.uniform(0, image_width)) for _ in range(cols - 1)]) + [image_width]
    left_y_coords = [0] + sorted([int(random.uniform(0, image_height)) for _ in range(rows - 1)]) + [image_height]
    right_y_coords = [0] + sorted([int(random.uniform(0, image_height)) for _ in range(rows - 1)]) + [image_height]
  else:
    top_x_coords =    [0] + [image_width /3, 2* image_width/3 ] + [image_width]
    bottom_x_coords = [0] + [image_width /3, 2* image_width/3 ] + [image_width]
    left_y_coords =   [0] + [image_height/3, 2* image_height/3] + [image_height]
    right_y_coords =  [0] + [image_height/3, 2* image_height/3] + [image_height]

  mesh = []
  sx = image_width/cols
  sy = image_height/rows
  for row in range(rows):
    for col in range(cols):

      TL_X = 0
      TL_Y = 1
      BL_X = 2
      BL_Y = 3
      BR_X = 4
      BR_Y = 5
      TR_X = 6
      TR_Y = 7

      prev_col = row * cols + col -1
      prev_row = (row-1) * cols + col
      top_left_x = mesh[prev_col][1][TR_X] if col else top_x_coords[col]
      bottom_left_x = mesh[prev_col][1][BR_X] if col else bottom_x_coords[col]
      bottom_right_x = bottom_x_coords[col+1]
      top_right_x = top_x_coords[col+1]

      top_left_y = mesh[prev_row][1][BL_Y] if row else left_y_coords[row]
      top_right_y = mesh[prev_row][1][BR_Y] if row else right_y_coords[row]
      bottom_left_y = left_y_coords[row + 1]
      bottom_right_y = right_y_coords[row + 1]

      x1 = col*sx
      y1 = row*sy
      x2 = (col+1)*sx
      y2 = (row+1)*sy

      #Source rectangle (unchanged)
      bbox = (int(x1), int(y1), int(x2), int(y2))

      # Create a quadrilateral representing the grid line
      quad = (
        top_left_x, top_left_y,
        bottom_left_x, bottom_left_y,
        bottom_right_x, bottom_right_y,
        top_right_x, top_right_y
        )

      iquad = tuple(map(int, quad))

      mesh.append((bbox,iquad))

  return mesh


def vertex_value(value, scale):
  """Returns the value plus or minus 10% of the scale.

  Args:
    value: The base value.
    scale: The scale to calculate the 10% from.

  Returns:
    The adjusted value.
  """
  offset = scale * 0.3 * random.uniform(-1, 1)  # Generate random offset within +/- 10%
  return value + offset

# Example usage
#result = value_plus_minus_10_percent(100, 50)
#print(result)  # Output: A value around 100, within +/- 5 of it (10% of 50)

def make_transparent(image, luminance_threshold):

  image = image.convert('RGBA')

  # Convert the image to a NumPy array
  image_array = np.array(image)

  # Create a mask based on luminance threshold (green channel)
  mask = image_array[:,:,1] > luminance_threshold

  # Set transparent pixels (alpha channel to 0) where mask is True
  image_array[mask, 3] = 0

  # Create a new image from the modified array
  transparent_image = Image.fromarray(image_array)

  return transparent_image

def blur(image, radius):

  # Apply Gaussian Blur

  width = int(image.width/3)
  height = int(image.height/3)
  cropped = image.crop((width,height,width*2,height*2))
  blurred_image = image.filter(ImageFilter.GaussianBlur(radius))
  blurred_image.paste(cropped,(width,height,width*2,height*2))

# return the blurred image
  return(blurred_image)


for file in range(files):
  # Open the image
  image = Image.open(f"{folder}{image_path}")
  im = make_transparent(image, 128)
  for _ in range(layers):
    # Apply the mesh transform
    #mesh = make_mesh(3,im)
    mesh = create_randomized_aligned_mesh(3,2,im.width,im.height)
    # Create a new image with the mesh
    out = im.transform(im.size, MeshTransform(mesh))
    blurred = blur(image, 5)
    mask = out if use_mask else None
    #draw the transformed image on the original using a mask
    image.paste(blurred)
    image.paste(out, None, mask)
  if test_mesh:
    draw_mesh(mesh, image)

  filename = f"{folder}mesh_image{file}.png"
  image.save(filename)
  image.show(filename)



In [19]:
def make_mask(width=200, height=200, border=5, rectangle=True):

  # Create a new image with a black background
  image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

  # Create a drawing object
  draw = ImageDraw.Draw(image)

  # Draw a circle with a gradient edge
  shape = draw.rectangle if rectangle else draw.elipse
  for i in range(border):
    alpha = int(255 * (i / border))  # Calculate alpha value for gradient
    shape(
        (
            i,
            i,
            width - i,
            height - i,
        ),
        fill=(255, 255, 255, alpha),
    )

  # return the image
  return(image)

image = make_mask()
image.save("mask.png")
image.show("mask.png")